In [1]:
import pandas as pd
import numpy as np
import glob
import tensorflow as tf
from matplotlib import pyplot as plt
from sklearn import metrics
from tensorflow.python.data import Dataset
pd.options.display.max_columns = 1200
pd.options.display.max_rows = 1200

ModuleNotFoundError: No module named 'tensorflow'

In [2]:
### Use a particular month:

# df = pd.read_csv('data/autoperf-2019-01.csv', header=0).sample(frac=1)
df = pd.read_csv('data/composed/mira_top_20.csv')

### OR use the full year

# li = []
# for filename in glob.glob("/home/luckierdodge/repos/jupyter-notebooks/data/autoperf-201*.csv"):
#     frame = pd.read_csv(filename, header=0)
#     li.append(frame)
# df = pd.concat(li, axis=0, ignore_index=True)

###

/home/luckierdodge/anaconda3/envs/autoperf_tf/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (876) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
### Filtering and cleaning the data before processing ###
# df.dropna(inplace=True);

###

counts = df.zero_execName.value_counts().to_dict()
df_filtered = pd.DataFrame()
for name in counts:
    if counts[name] >= 1000:
        df_filtered = df_filtered.append(df[df["zero_execName"] == name])

###

# df_filtered = df

###
df_filtered = df_filtered.sample(frac=1)

In [4]:
### Use this cell to explore columns ###

# for col in df_filtered.columns:
#     if 'time' in col or 'Time' in col:
#         print(col)


# df_filtered.zero_numRanks.describe()
# df_filtered.zero_execName.value_counts().to_dict()

In [19]:
### Use a particular subset of features:

selected_features = df_filtered[[
#     "zero_stdMpiTime",
    "gflops",
#     "gops",
#     "ram_store_bpop",
#     "ram_fetch_bpop",
#     "USED_CORE_HOURS",
#     "REQUESTED_CORE_HOURS"
#     "min_stdMpiTime",
#     "max_stdMpiTime",
#     "av_stdMpiTime",
#     "zero_numRanks",
#     "av_numProcessesOnNode",
#     "zero_elapsedTime",
#     "zero_numRanks",
#     "zero_elapsedCycles",
#     "zero_MPI_Barrier_callCount",
]]
features = selected_features.copy()

### OR use all of the features

# features = df.copy()
# features.drop(['zero_execName', 'run_date', 'zero_userName', 'zero_threadMode', 'min_userName', 'min_execName', 'min_threadMode', 'max_execName', 'max_userName', 'max_threadMode', 'av_execName', 'av_userName', 'av_threadMode'], axis=1, inplace=True)

###

# Factorize the targets for softmax classification (trying to predict executable name)
targets = pd.DataFrame()
targets['number'] = pd.factorize(df_filtered['zero_execName'], na_sentinel=-1)[0]
# targets['number'] = pd.factorize(df_filtered['zero_userName'], na_sentinel=-1)[0]

In [20]:
# Turn the pandas dataframe into tf.Keras friendly numpy arrays
def create_arrays(dataframe):
    array = []
    for col in dataframe.columns:
        array.append(dataframe[col].to_numpy())
    return array

In [21]:
# Separate our data into training and test sets

train_size = .8

train_examples = create_arrays(features.head(int(df_filtered.size * train_size)))
train_targets = create_arrays(targets.head(int(df_filtered.size * train_size)))
test_examples = create_arrays(features.tail(int(df_filtered.size * (1-train_size))))
test_targets = create_arrays(targets.tail(int(df_filtered.size * (1-train_size))))

In [22]:
# Create a TF.Keras model, taking into account the number of unique executables we're targetting in our softmax layer

### More complex model to tackle a full year

# model = tf.keras.models.Sequential([
#   tf.keras.layers.Dense(2048, activation='relu', input_shape=(1,)),
#   tf.keras.layers.Dropout(.2),
#   tf.keras.layers.Dense(1024, activation='relu'),
#   tf.keras.layers.Dropout(.2),
#   tf.keras.layers.Dense(targets.number.unique().size, activation='softmax')
# ])

### Single month, simplistic neural network

model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(128, activation='relu', input_shape=(1,)),
  tf.keras.layers.Dropout(.2),
  tf.keras.layers.Dense(targets.number.unique().size, activation='softmax')
])

###

In [26]:
model.compile(optimizer=tf.train.AdagradOptimizer(0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'],
            )

In [27]:
model.fit(train_examples, train_targets, epochs=20)

Epoch 1/20
88830/88830 [==============================] - 3s 38us/sample - loss: 1.2267 - acc: 0.5702
Epoch 2/20
88830/88830 [==============================] - 3s 36us/sample - loss: 1.1782 - acc: 0.5730
Epoch 3/20
88830/88830 [==============================] - 3s 36us/sample - loss: 1.1515 - acc: 0.5792
Epoch 4/20
88830/88830 [==============================] - 3s 36us/sample - loss: 1.1332 - acc: 0.5829
Epoch 5/20
88830/88830 [==============================] - 3s 36us/sample - loss: 1.1210 - acc: 0.5860
Epoch 6/20
88830/88830 [==============================] - 3s 36us/sample - loss: 1.1120 - acc: 0.5871
Epoch 7/20
88830/88830 [==============================] - 3s 36us/sample - loss: 1.1021 - acc: 0.5880
Epoch 8/20
88830/88830 [==============================] - 3s 36us/sample - loss: 1.0947 - acc: 0.5904
Epoch 9/20
88830/88830 [==============================] - 3s 36us/sample - loss: 1.0895 - acc: 0.5881
Epoch 10/20
88830/88830 [==============================] - 3s 36us/sample - loss: 

In [28]:
test_loss, test_acc = model.evaluate(test_examples, test_targets)
print('Test accuracy:', test_acc)

88830/88830 [==============================] - 2s 24us/sample - loss: 1.0245 - acc: 0.5795
Test accuracy: 0.5794664
